In [12]:
from src.model import lr, random_forest
from src.features import anf
from src.experiment.reports.latex_report import *
from src.experiment import Experiment
from src.data import Species, load_psi

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [9]:
human_test = load_psi(Species.human, True)
human_train = load_psi(Species.human, False)

# mouse_train_dataset = load_psi(Species.mouse, False)
# 
# yeast_test_dataset = load_psi(Species.yeast, True)
# yeast_train_dataset = load_psi(Species.yeast, False)

In [6]:
encoder = anf.Encoder()

In [10]:
lr_report = Experiment(lr.Factory(), human_test, human_train, encoder).run()

In [14]:
generate_latex_report(lr_report['test'], 'test', Path('lr_anf_human'), True)

note: Running TeX ...
note: Rerunning TeX because "test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `lr_anf_human/test.pdf` (27.12 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


In [15]:
rf_report = Experiment(random_forest.Factory(), human_test, human_train, encoder).run()

In [17]:
generate_latex_report(rf_report['test'], 'test', Path('rf_anf_human'), True)

note: Running TeX ...
note: Rerunning TeX because "test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `rf_anf_human/test.pdf` (26.09 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)
